## Import Ipyparallel first

In [ ]:
import ipyparallel

## Define client views

In [ ]:
clients = ipyparallel.Client('/home/snara025/.ipython/profile_myprofile/security/ipcontroller-client.json')#, url_file='/home/snara025/.ipython/profile_myprofile/security/ipcontroller-client.json', sshserver='192.168.233.126',

lview = clients.load_balanced_view()
dview = clients[:]

## Use cell magic to execute a given cell on all engines. 

* For instance, %%px executes the commands in the cell on all engines. The --local flag runs it on the local kernel as well.

In [ ]:
%%px --local 

from mvpa2.suite import *

import pandas as pd
import numpy as np
import math

In [ ]:
%%px --local

runs = np.arange(7,13,1)
run_type = ['test','train']
runs,run_type = zip(*itertools.product(runs,run_type)) #create a list to iterate over

## Define a function that can be used to send tasks to engines

In [ ]:
%%px --local

    
def get_betas_sub_02(runs, run_type):
    
    global conditions
    
    data_path = '/scratch/sanjay/GRT/fmri_data/testing/output/final/sub-02_{}_{}.nii' .format(run_type, runs)
    V1_mask = '/scratch/sanjay/GRT/fmri_data/testing/sub-02/sub-02_V1_full.nii' 
    dataset = fmri_dataset(samples = data_path,  mask = V1_mask) # get each volume file
    
    if len(dataset) > 226: # define a conditions vector with unique values
        conditions = np.arange(1,91,1)# for training runs
    else: 
        conditions = np.arange(1,46, 1)
    
    a = np.zeros(len(conditions)) # get stimulus onsets
    a[0] = 10 # i think this should always be ten
    for i in range(1,len(conditions),1):
        a[i] = a[i-1] + 6
        
    onsets = a
    
    voxels = np.asarray(dataset.samples) # get voxel timeseries into an array
    
    #run R1GLMS with FIR basis
    hrfs, betas = he.glm(conditions, onsets, 1.5, voxels, mode='r1glms', basis='fir', verbose=2, n_jobs=-1)
    
    nimg = map2nifti(dataset, data = betas) # reconvert  the estimated betas into a nifty file
    nimg.to_filename('/scratch/sanjay/GRT/deconvolution/sub-02_{}_{}_betas.nii' .format(run_type , runs))# save nifty file
    
   
    
    

## Send the tasks to the engines via LoadBalancedView

In [ ]:
output = lview.map(get_betas_sub_02, runs, run_type)
print output.wait_interactive()
output.get()